In [1]:
import sys
import matplotlib.pyplot as plt
import numpy as np    
import ROOT
import root_numpy as rtnp

import cygnus_lib as cy
import mylib as my
%matplotlib inline
import pandas as pd
from scipy.optimize import curve_fit
from scipy.stats import chisquare
import scipy.stats as stats
from itertools import combinations
import math
import os

from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
mybasepath = '/workarea/cloud-storage/mazzitel/'


Welcome to JupyROOT 6.22/06


In [12]:
runI          = [4061] 
run_ped       = 4062
#run_ped       = 4020
cimax         = 10
cimin         = 0 
tag           = 'LAB'
rescale       = 576
rescale       = 512
nsigma        = 0.5


#########################

m_image, s_image = cy.ped_(run_ped, path=mybasepath+'ped/')

#########################

th_image   = np.round(m_image + nsigma*s_image)
print ("light over Th: %.2f " % (th_image.sum()-m_image.sum()))
    
for nRi in range(len(runI)):
    try:
        print ('Open file: '+cy.s3_root_file(runI[nRi]), tag)
        f = ROOT.TFile.Open(cy.s3_root_file(runI[nRi], tag))
        print ('Find Keys: '+str(len(f.GetListOfKeys())))
        pic, wfm = cy.root_TH2_name(f)
        max_image = len(pic)
        max_wfm = len(wfm)
        print ("# of Images (TH2) Files: %d " % (max_image))
        print ("# of Waveform (TH2) Files: %d " % (max_wfm))

    except:
        print ("ERROR: No file %d" % (runI[nRi]))
        break
    # crea un db vuoto da riempire 
    df = pd.DataFrame(columns = ["iTr", "cluster_lable", "pixels", "photons", "ph_pixels", "x0start", "y0start", 
          "x0end", "y0end", "width", "height", "pearson"])
    # crea nome file di output
    file_out = (mybasepath+"out/dbscan_run%d_cmin_%d_cmax_%d_rescale_%d_nsigma_%.1f_ev_%d_ped_%d.plk.gz" % 
                (runI[nRi], cimin, cimax, rescale, nsigma, max_image, run_ped))
    for iTr in range(0, max_image): # max_image per tutte le imagini
        if iTr % 10 == 0: 
          # running & debug ...
          print ('Processing RUN: ', runI[nRi], 'Event: ', iTr)
          # end 

        image = rtnp.hist2array(f.Get(pic[iTr])).T

        rebin_image     = cy.rebin(image-m_image, (rescale, rescale))  
        rebin_th_image  = cy.rebin((th_image-m_image), (rescale, rescale))

        edges           = (rebin_image > rebin_th_image) & (rebin_image < cimax)         
        points          = np.array(np.nonzero(edges)).T.astype(float) 
        X_scaled        = StandardScaler().fit_transform(points) # 2D
        # X_scaled        = StandardScaler().fit_transform(cy.poit_3d(points, rebin_image)) # 3D
        dbscan   = DBSCAN(eps=0.1, min_samples = 20).fit(X_scaled) # eps=0.08 per 3D 0.04 2D
        clusters = dbscan.fit_predict(X_scaled)
        n_cluster = max(dbscan.labels_)
        print ("number of cluster: " +str(n_cluster), clusters) 
        for ic in range(0, n_cluster):
            ph = 0.
            yc = points[:,1][dbscan.labels_==ic]
            xc = points[:,0][dbscan.labels_==ic]
            ph, dim = cy.cluster_par(yc, xc, rebin_image)
            width, height, pearson = cy.confidence_ellipse_par(yc,xc)
            for j in range(0, dim):
                x=int(xc[j])
                y=int(yc[j])
                #ph += rebin_image[y,x]
                if j == 0:
                    x0start = x
                    y0start = y
            x0end = x
            y0end = y
            # print(iTr, ic, dim, ph, ph/dim,x0start,y0start,x0end, y0end, width, height,pearson)
            if abs(pearson)>0.9:
                fig = plt.figure(figsize=(5,5))
                plt.imshow(rebin_image, vmin=-5, vmax=30)
                yc = points[:,1][dbscan.labels_==ic]
                xc = points[:,0][dbscan.labels_==ic]
                plt.plot(yc,xc, 'r.', markersize=1)
                plt.show()
            df = df.append({"iTr":iTr, "cluster_lable":ic, "pixels":dim, "photons":ph, "ph_pixels":ph/dim, "x0start":x0start, "y0start":y0start, 
              "x0end":x0end, "y0end":y0end, "width":width, "height":height, "pearson":pearson},
                           ignore_index=True)
    df.to_pickle(file_out, compression='gzip')
    print ("out file", file_out)

light over Th: nan 
Open file: /workarea/cloud-storage/cygnus/Data/LAB/histograms_Run04061.root LAB
Find Keys: 200
# of Images (TH2) Files: 200 
# of Waveform (TH2) Files: 0 
Processing RUN:  4061 Event:  0


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:52: RuntimeWarning: invalid value encountered in greater


number of cluster: 0 [0 0 0 ... 0 0 0]


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:52: RuntimeWarning: invalid value encountered in greater


number of cluster: 2 [0 0 0 ... 0 0 0]


...

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:52: RuntimeWarning: invalid value encountered in greater


number of cluster: 0 [0 0 0 ... 0 0 0]


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:52: RuntimeWarning: invalid value encountered in greater


number of cluster: 0 [0 0 0 ... 0 0 0]


KeyboardInterrupt: 